# Book Recommendation Engine using KNN

In this challenge, you will create a book recommendation algorithm using K-Nearest Neighbors.

You will use the Book-Crossings dataset. This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users.

After importing and cleaning the data, use NearestNeighbors from sklearn.neighbors to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures the distance to determine the “closeness” of instances.

Create a function named get_recommends that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

--2024-02-11 07:23:04--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  15.7MB/s    in 1.6s    

2024-02-11 07:23:06 (15.7 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Check the dataset structure for Books
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [ ]:
# Check the dataset structure for Users ratings
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [ ]:
# Clean the data by remove the null data

# Check null data
print(df_books.isnull().sum())
print(df_ratings.isnull().sum())

# Remove null data from Books dataset
df_books.dropna(inplace=True)

#Check if there is any null data in books dataset
print(df_books.isnull().sum())

isbn      0
title     0
author    0
dtype: int64
user      0
isbn      0
rating    0
dtype: int64
isbn      0
title     0
author    0
dtype: int64


In [ ]:
# Remove users with less than 200 ratings from the dataset
user_ratings_count = df_ratings['user'].value_counts()
filtered_users = user_ratings_count[user_ratings_count < 200].index
df_ratings_filtered = df_ratings[~df_ratings['user'].isin(filtered_users)]
df_ratings_filtered.shape

(527556, 3)

In [ ]:
# Remove books with less than 100 ratings from the dataset
book_ratings_count = df_ratings['isbn'].value_counts()
filtered_books = book_ratings_count[book_ratings_count < 100].index
df_ratings_filtered = df_ratings_filtered[~df_ratings_filtered['isbn'].isin(filtered_books)]
df_ratings_filtered.shape

(49781, 3)

In [ ]:
# List of the recommended book
books = ["Where the Heart Is (Oprah's Book Club (Paperback))",
        "I'll Be Seeing You",
        "The Weight of Water",
        "The Surgeon",
        "I Know This Much Is True"]

for book in books:
  print(df_ratings_filtered.isbn.isin(df_books[df_books.title == book].isbn).sum())

183
75
49
57
77


In [ ]:
# Create a pivot table from the df_ratings_filtered dataframe, where:
# - Rows represent users
# - Columns represent ISBNs (books)
# - Values represent ratings given by users to books
# - Fill missing values with 0 (indicating no rating given)
# - Transpose the pivot table to make books as rows and users as columns
df = df_ratings_filtered.pivot_table(index=['user'], columns=['isbn'], values='rating').fillna(0).T

# Set the index of the pivot table to be the book titles
# This is achieved by joining the existing index with the 'title' column from df_books based on the ISBN
df.index = df.join(df_books.set_index('isbn'))['title']

# Sort the DataFrame by index (book titles)
df = df.sort_index()

In [ ]:
# Initialize the model with cosine distance metric
model = NearestNeighbors(metric='cosine')

# Fit the model to the data
model.fit(df.values)

NearestNeighbors(metric='cosine')

In [ ]:
# Specify the title of the book for which recommendations will be generated
title = 'The Queen of the Damned (Vampire Chronicles (Paperback))'

# Use the model to find the nearest neighbors (similar books) to the specified book
# Calculate the distances and indices of the nearest neighbors
# Note: We pass the feature values of the specified book to the model
distance, indice = model.kneighbors([df.loc[title].values], n_neighbors=6)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(title=""):
    try:
        # Try to retrieve the book by its title
        book = df.loc[title]
    except KeyError as e:
        # If the book does not exist, print a message and return
        print(f"The given book '{e}' does not exist")
        return

    # Find the nearest neighbors (similar books) using the model
    distances, indices = model.kneighbors([book.values], n_neighbors=6)

    # Create a DataFrame to store recommended books with their distances
    recommended_books = pd.DataFrame({
        'title': df.iloc[indices[0]].index.values,
        'distance': distances[0]
    })

    # Sort recommended books by distance in descending order and select the top 5
    recommended_books = recommended_books.sort_values(by='distance', ascending=False).head(5).values

    # Return the title of the book and the recommended books
    return [title, recommended_books]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", array([["I'll Be Seeing You", 0.8016210794448853],
       ['The Weight of Water', 0.7708583474159241],
       ['The Surgeon', 0.7699410915374756],
       ['I Know This Much Is True', 0.7677075266838074],
       ['The Lovely Bones: A Novel', 0.7234864234924316]], dtype=object)]
You passed the challenge! 🎉🎉🎉🎉🎉
